# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings("ignore")

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%time
uk_emb = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/Word Embedding/cc.uk.300.vec', limit=1000000)

CPU times: user 4min 25s, sys: 3.98 s, total: 4min 29s
Wall time: 4min 29s


In [4]:
ru_emb = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/Word Embedding/cc.ru.300.vec', limit=1000000)

In [5]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [6]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [7]:
ru_emb.most_similar([uk_emb["серпень"]])

[('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Недопустимость', 0.24435284733772278),
 ('персональ', 0.2368222028017044),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256804049015045),
 ('объемен', 0.23145240545272827),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [8]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [9]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2020-10-20 16:24:55--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2020-10-20 16:24:55--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2020-10-20 16:24:55--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Saving 

In [10]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2020-10-20 16:24:56--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2020-10-20 16:24:56--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2020-10-20 16:24:56--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Saving to

In [11]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [12]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [13]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [14]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august, topn=5)

[('апрель', 0.8554452061653137),
 ('июнь', 0.8427679538726807),
 ('март', 0.8408820629119873),
 ('сентябрь', 0.8370435237884521),
 ('февраль', 0.8334157466888428)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [15]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        most_similar = ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)
        if ru in [pair[0] for pair in most_similar]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [16]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [17]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [18]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [19]:
print(precision_top1)
print(precision_top5)

0.6298200514138818
0.8020565552699229


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [20]:
import numpy as np

In [21]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """

    u_matrix, sigma_matrix, vt_matrix = np.linalg.svd(X_train.T @ Y_train)
    mapping = u_matrix @ vt_matrix

    return mapping

In [22]:
W = learn_transform(X_train, Y_train)

In [23]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8263289928436279),
 ('сентябрь', 0.8077932596206665),
 ('июнь', 0.8059921860694885),
 ('март', 0.8046759366989136),
 ('октябрь', 0.801214873790741),
 ('ноябрь', 0.796485185623169),
 ('июль', 0.7954385280609131),
 ('февраль', 0.7941821217536926),
 ('август', 0.7939696311950684),
 ('декабрь', 0.7720677852630615)]

In [24]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6503856041131105
0.8097686375321337


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [25]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2020-10-20 16:27:30--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  1.87MB/s    in 0.9s    

2020-10-20 16:27:32 (1.87 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [26]:
!gzip -d ./uk.txt.gz

In [37]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [39]:
uk_corpus[:5]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n']

In [40]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [41]:
import nltk

tokenizer = nltk.tokenize.WordPunctTokenizer()

In [42]:
def tokenize(text):
    text = str(text).lower()
    return tokenizer.tokenize(text)

In [43]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    words = tokenize(sentence)
    translated = []

    for word in words:
        translated_embedding = np.matmul(uk_emb[word], W)
        translated.append(ru_emb.most_similar([translated_embedding])[0][0])

    return " ".join(translated)

In [44]:
assert translate(".") == "."
assert translate("1 , 11") == "1 , 11"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

Let's have a look at translations from this model.

In [45]:
for sent in uk_corpus[1:6]:
    print('Original sentence: {}'.format(sent[:-1]))
    print('Translated sentence: {}'.format(translate(sent) + '\n'))

Original sentence: Він наказав мені негайно вийти з кімнати.
Translated sentence: он велел мне немедленно выйти со комнаты .

Original sentence: Як би ти не намагався, ти не вивчиш англійську за два-три місяці.
Translated sentence: как бы ты не пытался , ты не выучишь английский за два – три месяца .

Original sentence: Поки я не подзвонив, він не прийшов.
Translated sentence: пока мной не позвонил , он не пришел .

Original sentence: У всесвіті багато галактик.
Translated sentence: во вселенной много галактик .

Original sentence: Вона приймає душ щоранку.
Translated sentence: она принимает души утрам .



They look not bad, however, there is a big problem OOV words. Let's translate each of them to a special **UNK** token. At least, it will allow us to see how much OOV words do we have in our data.

In [48]:
def translate_with_unk(sentence):
    words = tokenize(sentence)
    translated = []

    for word in words:
        if word not in uk_emb.vocab:
            translated.append('UNK')
        else:
            translated_embedding = np.matmul(uk_emb[word], W)
            translated.append(ru_emb.most_similar([translated_embedding])[0][0])

    return " ".join(translated)

In [47]:
for sent in uk_corpus[::100]:
    print('Original sentence: {}'.format(sent[:-1]))
    print('Translated sentence: {}'.format(translate_with_unk(sent) + '\n'))

Original sentence: Я вже закінчу коледж, коли ви вернетеся з Америки.
Translated sentence: мной уже закончу колледж , когда мы UNK со америки .

Original sentence: У мене є мрія.
Translated sentence: во меня то мечта .

Original sentence: Я знаю, що ти багатий.
Translated sentence: мной знаю , что ты богатый .

Original sentence: Коли Ви народилися?
Translated sentence: когда мы родились ?

Original sentence: Один, два, три, чотири, п'ять, шість, сім, вісім, дев'ять, десять.
Translated sentence: один , два , три , четыре , аш со пять , восемь , семь , восемь , девять со пять , десять .

Original sentence: Я би з задоволенням написав сотні речень в Tatoeb’і, але в мене є справи.
Translated sentence: мной бы со удовольствием сочинил сотни сложноподчинённых во UNK со и , конечно во меня то дела .

Original sentence: З юбілеєм Олексія Дударева привітав Президент Білорусі Олександр Лукашенко.
Translated sentence: со UNK UNK UNK поприветствовал президент Белоруси азаров UNK .

Original sente

Great! 
See second notebook for the Neural Machine Translation assignment.